<a href="https://colab.research.google.com/github/GKouv89/tt8/blob/tempNavigation/dataScripts/excelToCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
path = "/content/drive/MyDrive/TT8/Sociodrama 1/All data - superepisodes"
target_path = "/content/drive/MyDrive/TT8/CSV files"

In [ ]:
import fnmatch
import os
import pandas as pd

In [ ]:
# Read first file
file_list = os.listdir(path)

In [ ]:
import pandas as pd
file_path = path + '/' + file_list[0]
file_name_no_ending = file_list[0].replace('.xlsx', '')
new_file_path = target_path + '/' + file_name_no_ending + '.csv'
myfile = pd.read_excel(file_path)
myfile.to_csv(new_file_path, index = None, header=True)

In [ ]:
# First, make folders for all scenes
# Then, move data from all data folder, to respective scene folders, with correct format
# First, find scene number. Following naming convention, its the number right after the Scene word
def findSceneNo(name, from_file=True):
  scene_no = name.split("Scene")
  if from_file:
    # But the scene number could be 1, 2 or 3 digits long. So let's remove the part that's after the biometric
    # Which biometric is it? So far, 3 choices. 
    temp = scene_no[1].find("HR")
    if(temp == -1):
      temp = scene_no[1].find("SC")
      if(temp == -1):
        return scene_no[1].split("Temp")[0]
      else:
        return scene_no[1].split("SC")[0]
    else:
      return scene_no[1].split("HR")[0]
  else:
    return scene_no[1]

In [ ]:
# store a list of all the lists that we have created folders for
scenesList = []

In [ ]:
def convertToCSV(file_list):
  for file_name in file_list:
    scene_no = findSceneNo(file_name)
    folder_name = "Scene" + scene_no
    folder = os.path.join(target_path, folder_name)
    if int(scene_no) not in scenesList:
      scenesList.append(int(scene_no))
      os.mkdir(folder)
    # Create file
    myfile = pd.read_excel(os.path.join(path, file_name))
    file_name_new = file_name.replace('.xlsx', '') + '.csv'
    myfile.to_csv(os.path.join(folder, file_name_new), index = None, header=True)

In [ ]:
convertToCSV(file_list)

In [ ]:
def findSection(file):
  print("File: ", file)
  first_occ = file.find("Introduction")
  if first_occ == -1:
    first_occ = file.find("Section")
    if first_occ == -1:
      return "Conclusion"
    else:
      # Find section number, too
      return file.split("Section")[1].split(".")[0]
  else:
    return "Introduction"


def concatDiffSectionsBiometric(file_list, biometric, scene_dir, scene_no):
  # First, find all files in the list that have a specific biometric
  sensor_files = fnmatch.filter(file_list, "*{}*".format(biometric))
  print("Biometric: {}, sensor files: {}".format(biometric, sensor_files))
  # Types of section names could be something like 
  # Introduction
  # Section X-Y
  # Conclusion
  # These files should be two. Find out the section.
  if len(sensor_files) == 2:
    sections = []
    sections.append(findSection(sensor_files[0]))
    sections.append(findSection(sensor_files[1]))
    print("Sections for scene {} are {} and {}".format(scene_no, sections[0], sections[1]))
    # Compare sections. Do we have a conclusion or intro? things become easier
    if "Introduction" in sections:
      first_section = sections.index("Introduction")
      [second_section] = [ind for ind, x in enumerate(sections) if ind != first_section]
      print("Section {} comes before section {}.".format(sections[first_section], sections[second_section]))
    elif "Conclusion" in sections:      
      second_section = sections.index("Conclusion")
      [first_section] = [ind for ind, x in enumerate(sections) if ind != second_section]
      print("Section {} comes before section {}.".format(sections[first_section], sections[second_section]))
    else:
      print("Both sections are middle sections")
      # Both sections X1-Y1 and X2-Y2, we must compare
      section_a = int(sections[0].split("-")[0])
      section_b = int(sections[1].split("-")[0])
      if section_a != section_b:
        first_section = 0 if section_a < section_b else 1
        [second_section] = [ind for ind, x in enumerate(sections) if ind != first_section]
        print("Section {} comes before section {}.".format(sections[first_section], sections[second_section]))
      else:
        # Same section, different subsections. Compare
        subsection_a = int(sections[0].split("-")[1])
        subsection_b = int(sections[1].split("-")[1])
        print(subsection_a)
        print(subsection_b)
        if subsection_a != subsection_b:
          print("blah")
          first_section = 0 if subsection_a < subsection_b else 1
          [second_section] = [ind for ind, x in enumerate(sections) if ind != first_section]
          print("Section {} comes before section {}.".format(sections[first_section], sections[second_section]))
        else:
          print("ERROR. FILES: ", sensor_files)
    # Now that we know which section is first and which is second, we concatenate the file to a larger one.
    first_file = pd.read_csv(os.path.join(scene_dir, sensor_files[first_section]), header=None)
    second_file = pd.read_csv(os.path.join(scene_dir, sensor_files[second_section]), header=None)
    temp_frame = pd.concat([first_file, second_file], ignore_index=True)
    return temp_frame
  else:
    print("Error. Files: ", sensor_files)


def concatDiffSections(file_list, scene_dir, scene_no, sensor_name):
  new_file_list = []
  for biometric in ["HR", "SC", "Temp"]:
    new_file_list.append(concatDiffSectionsBiometric(file_list, biometric, scene_dir, sensor_name))
  return new_file_list

In [ ]:
def performConcatenation(files, scene_no, sensor_name, scene_dir):
  temp_frame = pd.concat(files, axis=1)
  concat_file_name = "Scene{}All{}.csv".format(scene_no, sensor_name)
  # print(concat_file_name)
  temp_path = os.path.join(dir_path, concat_file_name)
  temp_frame.to_csv(temp_path, index=False, header=['HR', 'SC', 'Temp'])
  # Reread file to add header and re-export
  # temp_frame = pd.read_csv(temp_path, names=['HR', 'SC', 'Temp'])
  # temp_frame.to_csv(temp_path, index=False)

def concatCSV(file_list, sensor_name, scene_dir):
  files = []
  scene_no = findSceneNo(scene_dir, from_file=False)
  file_list.sort()
  # print(file_list)
  # This is useful to avoid certain scenes that belong to two sessions
  if len(file_list) > 3:
    print("Scene {} is split between sections.".format(scene_no))
    files = concatDiffSections(file_list, scene_dir, scene_no, sensor_name)
    performConcatenation(files, scene_no, sensor_name, scene_dir)
  elif len(file_list) == 3:
    print('hi1')
    for ff in file_list:
      print('ff: {}'.format(ff))
      files.append(pd.read_csv(os.path.join(scene_dir, ff), header=None))
    performConcatenation(files, scene_no, sensor_name, scene_dir)

In [ ]:
# Find all files that have to do with a specific sensor for a specific scene
dir_list = fnmatch.filter(os.listdir(target_path), "Scene*")
for dir in dir_list:
  for i in range(1, 7):
    sensor_name = "Sensor" + str(i)
    dir_path = os.path.join(target_path, dir)
    if os.path.isdir(dir_path):
      file_list_min = fnmatch.filter(os.listdir(dir_path), "*{}*".format(sensor_name))
      file_list_already_done = fnmatch.filter(file_list_min, "*All*")
      file_list_min = [x for x in file_list_min if x not in file_list_already_done]
      concatCSV(file_list_min, sensor_name, dir_path)

hi1
ff: Scene22HRSensor2-Section 2-2.csv
ff: Scene22SCSensor2-Section 2-2.csv
ff: Scene22TempSensor2-Section 2-2.csv
hi1
ff: Scene22HRSensor3-Section 2-2.csv
ff: Scene22SCSensor3-Section 2-2.csv
ff: Scene22TempSensor3-Section 2-2.csv
hi1
ff: Scene22HRSensor4-Section 2-2.csv
ff: Scene22SCSensor4-Section 2-2.csv
ff: Scene22TempSensor4-Section 2-2.csv
hi1
ff: Scene22HRSensor5-Section 2-2.csv
ff: Scene22SCSensor5-Section 2-2.csv
ff: Scene22TempSensor5-Section 2-2.csv
hi1
ff: Scene22HRSensor6-Section 2-2.csv
ff: Scene22SCSensor6-Section 2-2.csv
ff: Scene22TempSensor6-Section 2-2.csv
hi1
ff: Scene48HRSensor2-Section 4-2.csv
ff: Scene48SCSensor2-Section 4-2.csv
ff: Scene48TempSensor2-Section 4-2.csv
hi1
ff: Scene48HRSensor3-Section 4-2.csv
ff: Scene48SCSensor3-Section 4-2.csv
ff: Scene48TempSensor3-Section 4-2.csv
hi1
ff: Scene48HRSensor4-Section 4-2.csv
ff: Scene48SCSensor4-Section 4-2.csv
ff: Scene48TempSensor4-Section 4-2.csv
hi1
ff: Scene48HRSensor5-Section 4-2.csv
ff: Scene48SCSensor5-Se

In [27]:
# Delete initial csvs and keep only "All"
from os import walk
for (root, dirs, all_files) in walk(target_path):
  all_files_min = fnmatch.filter(all_files, "*All*")
  all_files_diff = [x for x in all_files if x not in all_files_min]
  for name in all_files_diff:
    # print(os.path.join(root, name))
    os.remove(os.path.join(root, name))

for (root, dirs, all_files) in walk(target_path):
  for name in all_files:
    print(os.path.join(root,name))
  print("*************")

*************
/content/drive/MyDrive/TT8/CSV files/Scene22/Scene22AllSensor2.csv
/content/drive/MyDrive/TT8/CSV files/Scene22/Scene22AllSensor3.csv
/content/drive/MyDrive/TT8/CSV files/Scene22/Scene22AllSensor4.csv
/content/drive/MyDrive/TT8/CSV files/Scene22/Scene22AllSensor5.csv
/content/drive/MyDrive/TT8/CSV files/Scene22/Scene22AllSensor6.csv
*************
/content/drive/MyDrive/TT8/CSV files/Scene48/Scene48AllSensor2.csv
/content/drive/MyDrive/TT8/CSV files/Scene48/Scene48AllSensor3.csv
/content/drive/MyDrive/TT8/CSV files/Scene48/Scene48AllSensor4.csv
/content/drive/MyDrive/TT8/CSV files/Scene48/Scene48AllSensor5.csv
/content/drive/MyDrive/TT8/CSV files/Scene48/Scene48AllSensor6.csv
*************
/content/drive/MyDrive/TT8/CSV files/Scene18/Scene18AllSensor2.csv
/content/drive/MyDrive/TT8/CSV files/Scene18/Scene18AllSensor3.csv
/content/drive/MyDrive/TT8/CSV files/Scene18/Scene18AllSensor4.csv
/content/drive/MyDrive/TT8/CSV files/Scene18/Scene18AllSensor5.csv
/content/drive/MyDri